# Computational Theory 

In [667]:
# for numerical data and methods 
import numpy as np 

## Problem 1: Binary Words and Operations


The `numpy.int32()` constructor ([see official documentation] https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32) ensures that values are stored and treated as 32-bit signed integers in NumPy.

In [668]:
def parity(x,y,z):
    """Calculate the parity of three 32-bit integers"""
    # Ensure inputs are treated as 32-bit integers as per SHA-1 specification.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32
    x = np.int32(x)
    y = np.int32(y)
    z = np.int32(z)
    
    # Calculate the bitwise XOR operation on the three 32-bit values.
    parity_output = np.int32(x ^ y ^ z)
    # Return the result.
    return parity_output

### Parity Function
The `parity` function calculates the bitwise XOR of three 32-bit integers, as defined in the SHA-1 algorithm. For each bit position, it returns a result of 1 when an odd number of the input values is set to 1, and 0 otherwise. This is implemented as x ^ y ^ z.

In [669]:
# Test parity function 
# Test 1: All zeros
print("TEST 1: parity(0, 0, 0)")
test_result = parity(0, 0, 0)
expected_result = 0
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 2: One non-zero input
print("\nTEST 2: parity(1, 0, 0)")
test_result = parity(1, 0, 0)
expected_result = 1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 3: Two non-zero inputs
print("\nTEST 3: parity(1, 1, 0)")
test_result = parity(1, 1, 0)
expected_result = 0
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 4: Negative numbers
print("\nTEST 4: parity(-1, -2, -3)")
test_result = parity(-1, -2, -3)
expected_result = -4
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 5: Mixed inputs
print("\nTEST 5: parity(2, -4, 5)")
test_result = parity(2, -4, 5)
expected_result = -5
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 6: All ones
print("\nTEST 6: parity(1, 1, 1)")
test_result = parity(1, 1, 1)
expected_result = 1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 7: Three negative inputs
print("\nTEST 7: parity(-1, -1, -1)")
test_result = parity(-1, -1, -1)
expected_result = -1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

TEST 1: parity(0, 0, 0)
Result :   0 
Expected :  0 
Correct :  True

TEST 2: parity(1, 0, 0)
Result :   1 
Expected :  1 
Correct :  True

TEST 3: parity(1, 1, 0)
Result :   0 
Expected :  0 
Correct :  True

TEST 4: parity(-1, -2, -3)
Result :   -4 
Expected :  -4 
Correct :  True

TEST 5: parity(2, -4, 5)
Result :   -5 
Expected :  -5 
Correct :  True

TEST 6: parity(1, 1, 1)
Result :   1 
Expected :  1 
Correct :  True

TEST 7: parity(-1, -1, -1)
Result :   -1 
Expected :  -1 
Correct :  True


In [670]:
def ch(x, y, z):
    """Calculate the choose function for three 32-bit integers"""
    # Ensure inputs are treated as 32-bit integers as per SHA-1 specification.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32
    x = np.int32(x)
    y = np.int32(y)
    z = np.int32(z)
    
    # Bitwise AND of x and y.
    bitwise_and = (x & y)
    # Bitwise complement of x, then AND with z.
    bitwise_complement_and = (~x) & z
    # Calculate the bitwise XOR operation of the two results and cast to 32-bit integer.
    ch_output = np.int32(bitwise_and ^ bitwise_complement_and)
    # Return the result.
    return ch_output 

### Choose Function
The `choose` function, used in the SHA-224 and SHA-256 algorithms, calculates a conditional selection among three 32-bit integers. For each bit position, it returns the bit from y if the bit in x is 1, or the bit from z if the bit in x is 0. This is implemented as (x & y) ^(~x & z).

In [671]:
# Test ch function 

# Test 1: All zeros
print("TEST 1: ch(0, 0, 0)")
test_result = ch(0, 0, 0)
expected_result = 0
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 2: All ones
print("\nTEST 2: ch(1, 1, 1)")
test_result = ch(1, 1, 1)
expected_result = 1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 3: Only z is non-zero
print("\nTEST 3: ch(0, 0, 1)")
test_result = ch(0, 0, 1)
expected_result = 1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 4: Only z is zero
print("\nTEST 4: ch(1, 1, 0)")
test_result = ch(1, 1, 0)
expected_result = 1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 5: All negative ones
print("\nTEST 5: ch(-1, -1, -1)")
test_result = ch(-1, -1, -1)
expected_result = -1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 6: Negative and positive numbers 
print("\nTEST 6: ch(-1, 0, 1)")
test_result = ch(-1, 0, 1)
expected_result = 0
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)


TEST 1: ch(0, 0, 0)
Result :   0 
Expected :  0 
Correct :  True

TEST 2: ch(1, 1, 1)
Result :   1 
Expected :  1 
Correct :  True

TEST 3: ch(0, 0, 1)
Result :   1 
Expected :  1 
Correct :  True

TEST 4: ch(1, 1, 0)
Result :   1 
Expected :  1 
Correct :  True

TEST 5: ch(-1, -1, -1)
Result :   -1 
Expected :  -1 
Correct :  True

TEST 6: ch(-1, 0, 1)
Result :   0 
Expected :  0 
Correct :  True


In [672]:
def maj(x, y, z):
    """Calculate the majority value of three 32-bit integers."""
    # The np.int32() constructor ensures inputs are treated as 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32
    x = np.int32(x)
    y = np.int32(y)
    z = np.int32(z)

    # Bitwise AND of x and y.
    bitwise_and_x_y= (x & y)   
    # Bitwise AND of x and z.
    bitwise_and_x_z= (x & z) 
    # Bitwise AND of y and z.   
    bitwise_and_y_z= (y & z)   
    # Calculate the bitwise XOR operation on the three results and cast to 32-bit integer.
    maj_output = np.int32(bitwise_and_x_y ^ bitwise_and_x_z ^ bitwise_and_y_z)
    # Return the result
    return maj_output 

### Majority Function
The `maj` function, used in the SHA-224 and SHA-256 algorithms, calculates the majority value among three 32-bit integers. For each bit position, it returns 1 if at least two of the three bits among x, y, and z are 1, and 0 otherwise. This is implemented as (x & y) ^ (x & z) ^ (y & z).

In [673]:
# Test maj function
# Test 1: All zeros
print("TEST 1: maj(0, 0, 0)")
test_result = maj(0, 0, 0)
expected_result = 0
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 2: All ones
print("\nTEST 2: maj(1, 1, 1)")
test_result = maj(1, 1, 1)
expected_result = 1
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 3: One non-zero input
print("\nTEST 3: maj(1, 0, 0)")
test_result = maj(1, 0, 0)
expected_result = 0
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 4: Two non-zero inputs
print("\nTEST 4: maj(2, 2, 0)")
test_result = maj(2, 2, 0)
expected_result = 2
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 5: All different numbers
print("\nTEST 5: maj(3, 5, 7)")
test_result = maj(3, 5, 7)
expected_result = 7
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

TEST 1: maj(0, 0, 0)
Result :   0 
Expected :  0 
Correct :  True

TEST 2: maj(1, 1, 1)
Result :   1 
Expected :  1 
Correct :  True

TEST 3: maj(1, 0, 0)
Result :   0 
Expected :  0 
Correct :  True

TEST 4: maj(2, 2, 0)
Result :   2 
Expected :  2 
Correct :  True

TEST 5: maj(3, 5, 7)
Result :   7 
Expected :  7 
Correct :  True


## Problem 2: Fractional Parts of Cube Roots 


In [674]:
# Write a function called primes(n) that generates the first n prime numbers.

# Use the function to calculate the cube root of the first 64 primes.

# For each cube root, extract the first thirty-two bits of the fractional part.

# Display the result in hexadecimal.

# Test the results against what is in the Secure Hash Standard.

## Problem 3: Padding


In [675]:
# Write a generator function block_parse(msg) that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 

## Problem 4: Hashes


In [676]:
# Write a function hash(current, block) that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

## Problem 5: Passwords


In [677]:
# The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. As strings, they were encoded using UTF-8. Determine the passwords and explain how you found them. 

## End